# Notebook to explore the data for the clustering step

In [0]:
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", 1024)

In [0]:
from pyspark.sql import functions as sf
input_container_path = f"wasbs://energypatents@challengebasf.blob.core.windows.net"
input_blob_folder = f"{input_container_path}/output_data_v1/"
df = spark.read.parquet(input_blob_folder)
df = df.select("_file", "prediction", "english_text_features")
df = df.filter(sf.col("prediction") == 1)
df = df.drop("prediction")

In [0]:
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml.pipeline import Pipeline

min_df = 0.10
max_df = 0.80
cv = CountVectorizer(inputCol="english_text_features", outputCol="vectors", minDF=min_df, maxDF=max_df)
idf = IDF(inputCol="vectors", outputCol="features")
pipe = Pipeline(stages=[cv, idf])
model = pipe.fit(df)
result_idf = model.transform(df)

In [0]:
result_idf = result_idf.cache()

In [0]:
from pyspark.ml.clustering import KMeans
n = 10
list_n = [4,6,8,10,15,20]
for n in list_n:
  k_means = KMeans(k=n, maxIter=100, seed=18,  distanceMeasure="cosine")
  k_means_model = k_means.fit(result_idf)
  print(n)
  print(k_means_model.summary.clusterSizes)
  print(k_means_model.summary.trainingCost)

4
[23288, 31771, 2148, 47417]
74833.74445942616
6
[16013, 22575, 2037, 35420, 14978, 13601]
70863.43324688774
8
[14454, 14668, 9461, 6545, 11164, 11518, 11796, 25018]
68311.79210853708
10
[14422, 1531, 8631, 6519, 23825, 11173, 11172, 3483, 9765, 14103]
67191.43032841313
15
[10377, 3975, 7965, 1531, 6002, 6868, 11984, 8397, 4359, 9447, 2608, 2876, 11633, 10336, 6266]
64150.26169809047
20
[11622, 6212, 5543, 6458, 5592, 2449, 6322, 2688, 7573, 3428, 7629, 6823, 1433, 1477, 3929, 3771, 7767, 3703, 5526, 4679]
61982.25656148839

We choose n=10 (elbow rule)

In [0]:
n = 10
k_means = KMeans(k=n, maxIter=100, seed=18,  distanceMeasure="cosine")
k_means_model = k_means.fit(result_idf)
print(n)
print(k_means_model.summary.clusterSizes)
print(k_means_model.summary.trainingCost)

10
[14422, 1531, 8631, 6519, 23825, 11173, 11172, 3483, 9765, 14103]
67191.43032841313

In [0]:
result_final = k_means_model.transform(result_idf)

In [0]:
result_final.printSchema()

root
-- _file: string (nullable = true)
-- english_text_features: array (nullable = true)
 |-- element: string (containsNull = true)
-- vectors: vector (nullable = true)
-- features: vector (nullable = true)
-- prediction: integer (nullable = false)

In [0]:
result_final = result_final.cache()

In [0]:
result_final.select("prediction").drop_duplicates().show()

+----------+
prediction|
+----------+
 1|
 6|
 3|
 5|
 9|
 4|
 8|
 7|
 2|
 0|
+----------+

In [0]:
centers = k_means_model.clusterCenters()
print(f"Centers: {centers}")
summary = k_means_model.summary
summary.k

Centers: [array([0.01119377, 0.01919199, 0.02955908, ..., 0.00274494, 0.00183779,
 0.00104622]), array([0.0114327 , 0.01282585, 0.01797786, ..., 0.00126251, 0.00101318,
 0.00083675]), array([0.03487552, 0.02277332, 0.04881191, ..., 0.002124 , 0.00169292,
 0.0013483 ]), array([0.03481729, 0.01340454, 0.01206047, ..., 0.00048756, 0.00067612,
 0.00134186]), array([0.02410142, 0.0282665 , 0.04911831, ..., 0.0029531 , 0.00290264,
 0.00368588]), array([0.07819099, 0.02327563, 0.04775551, ..., 0.0011735 , 0.00117962,
 0.00208464]), array([0.02797318, 0.03116855, 0.02885352, ..., 0.00223084, 0.00177028,
 0.00209206]), array([0.00901672, 0.01543703, 0.01201878, ..., 0.00214592, 0.00140986,
 0.00183529]), array([0.03268966, 0.02284195, 0.02146522, ..., 0.00158648, 0.00167068,
 0.00344434]), array([0.19709712, 0.04757033, 0.04075381, ..., 0.00258698, 0.00288115,
 0.00185498])]
Out[17]: 10

In [0]:
summary.clusterSizes

Out[18]: [14422, 1531, 8631, 6519, 23825, 11173, 11172, 3483, 9765, 14103]

In [0]:
import pandas as pd
vocab = model.stages[0].vocabulary
list_dfs = []
for num_cluster in range(n):
  df = pd.DataFrame({"word":vocab, "value": centers[num_cluster]})
  df.loc[:, "cluster"] = num_cluster
  list_dfs.append(df)
total_df = pd.concat(list_dfs, axis=1)

In [0]:
total_df.head()

,word,value,cluster,word,value,cluster,word,value,cluster,word,value,cluster,word,value,cluster,word,value,cluster,word,value,cluster,word,value,cluster,word,value,cluster,word,value,cluster
0,signal,0.011194,0,signal,0.011433,1,signal,0.034876,2,signal,0.034817,3,signal,0.024101,4,signal,0.078191,5,signal,0.027973,6,signal,0.009017,7,signal,0.032690,8,signal,0.197097,9
1,b,0.019192,0,b,0.012826,1,b,0.022773,2,b,0.013405,3,b,0.028266,4,b,0.023276,5,b,0.031169,6,b,0.015437,7,b,0.022842,8,b,0.047570,9
2,unit,0.029559,0,unit,0.017978,1,unit,0.048812,2,unit,0.012060,3,unit,0.049118,4,unit,0.047756,5,unit,0.028854,6,unit,0.012019,7,unit,0.021465,8,unit,0.040754,9
3,module,0.028371,0,module,0.020164,1,module,0.052605,2,module,0.018422,3,module,0.127349,4,module,0.042245,5,module,0.060328,6,module,0.035532,7,module,0.065659,8,module,0.066607,9
4,service,0.006376,0,service,0.020728,1,service,0.009970,2,service,0.027288,3,service,0.123728,4,service,0.055240,5,service,0.061019,6,service,0.085975,7,service,0.090517,8,service,0.014016,9


In [0]:
total_df.to_excel("cluster_centers.xlsx", header=True, index=False)

In [0]:
%fs cp file:/databricks/driver/cluster_centers.xlsx dbfs:/FileStore/cluster_centers.xlsx

res1: Boolean = true

Analysing the words with most value in each topic we can identify:
1 - Physics  
2 - Gaming  
3 - Photography  
4 - Mobile networks  
5 - Computers/cloud IT  
6 - Mobile networks  
7 - Networking  
8 - Video streaming  
9 - Mobile